# Класифікація спаму: Naive Bayes

**Мультиноміальна модель подій**

Ми застосуємо наївний Баєсівський класифікатор зі згладжуванням Лапласа для навчання спам-фільтру на основі даних [SpamAssassin Public Corpus](http://spamassassin.apache.org/publiccorpus/).

Заповніть пропущений код (позначено коментарями) та визначте точність передбачення. У вас повинен вийти кращий результат, ніж при моделі багатовимірного розподілу Бернуллі.

In [23]:
import json

In [24]:
import numpy as np

from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as progressbar

## Завантаження даних

Щоб краще зрозуміти, яким чином були очищені дані, див. [`spam-data-preparation.ipynb`](spam-data-preparation.ipynb).

In [25]:
def load_json_from_file(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)

In [26]:
emails_tokenized_ham = load_json_from_file("emails-tokenized-ham.json")
emails_tokenized_spam = load_json_from_file("emails-tokenized-spam.json")

In [27]:
vocab = load_json_from_file("vocab.json")

## Кодування даних

Представте кожен лист як $n$-вимірний вектор $\left[ x_1, x_2, ..., x_n \right]$, де $x_i$ — це індекс $i$-го слова даного листа у словнику $V$, а $n$ — кількість слів у листі.

Наприклад, лист _"Buy gold watches. Buy now."_ міг би бути закодований так: $\left[ 3953, 11890, 32213, 3953, 20330 \right]$.

In [28]:
def email_to_vector_multinomial(email_words, vocab):
    # =============== TODO: Your code here ===============
    # Build a feature vector for a single email using the
    # multinomial event model.
    res = np.zeros(len(email_words), dtype="int64")
    for i in range(len(email_words)):
        res[i] = vocab[email_words[i]]
    return res
    # ====================================================

Тепер закодуємо всі листи:

In [29]:
X = [
    email_to_vector_multinomial(email, vocab)
    for email in emails_tokenized_ham + emails_tokenized_spam
]

In [30]:
y = np.array([0] * len(emails_tokenized_ham) + [1] * len(emails_tokenized_spam))

Поглянемо на кілька випадкових листів:

In [31]:
sample_emails = [emails_tokenized_ham[10], emails_tokenized_ham[70]]

In [32]:
for email in sample_emails:
    print(email)
    print()

['hello', 'seen', 'discuss', 'articl', 'approach', 'thank', 'httpaddress', 'hell', 'rule', 'tri', 'accomplish', 'someth', 'thoma', 'alva', 'edison', 'sf', 'net', 'email', 'sponsor', 'osdn', 'tire', 'old', 'cell', 'phone', 'get', 'new', 'free', 'httpaddress', 'spamassassin', 'devel', 'mail', 'list', 'emailaddress', 'httpaddress']

['fri', 'number', 'aug', 'number', 'tom', 'wrote', 'xvid', 'number', 'project', 'make', 'gpl', 'divx', 'codec', 'sigma', 'design', 'number', 'sorri', 'sigma', 'design', 'number', 'number', 'httpaddress']



In [33]:
for email in sample_emails:
    email_vec = email_to_vector_multinomial(email, vocab)
    
    print("Email vector:", email_vec)
    print("Dimensionality:", email_vec.shape)
    print()

Email vector: [12866 26186  7632  1574  1361 29410 13468 12862 25408 30214   173 27396
 29564   872  8567 26411 19758  8849 27722 21116 29770 20748  4549 22215
 11528 19855 10871 13468 27540  7314 17535 16947  8851 13468]
Dimensionality: (34,)

Email vector: [10946 20419  1845 20419 29891 33008 33256 20419 23298 17594 12021  7779
  5370 26756  7232 20419 27443 26756  7232 20419 20419 13468]
Dimensionality: (22,)



## Розділення вибірок

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [35]:
print("# Train:", len(X_train))
print("# Test: ", len(X_test))

# Train: 4987
# Test:  555


## Навчання наївного Баєсового класифікатора

Підрахуйте сумарну кількість слів у ham- і spam-листах відповідно.

In [36]:
# =============== TODO: Your code here ===============
# Count the total number of words in ham and spam emails.
# Store these counts into the two variables defined below.
# print(len(y_train))
# print(sum(num == 1 for num in y_train))
# print(sum(num == 0 for num in y_train))

# ham_total_words_train = sum(num == 0 for num in y_train)
# spam_total_words_train = sum(num == 1 for num in y_train)

ham_total_words_train = 0
spam_total_words_train = 0

for i in range(len(X_train)):
    if (y_train[i] == 1):
        spam_total_words_train += len(X_train[i]) 
    else:
        ham_total_words_train += len(X_train[i]) 
# ====================================================

Тепер обчисліть апріорні імовірності для класів ham і spam. Зауважте, що добуток імовірностей може переповнити тип даних змінної, тому ми будемо використовувати логарифми.

In [37]:
# =============== TODO: Your code here ===============
# Compute the class priors for ham and spam emails.
spam_total_emails_train = 0
ham_total_emails_train = 0

for value in y_train:
    if (value == 1):
        spam_total_emails_train+=1
    else:
        ham_total_emails_train+=1

ham_log_prior = np.log(ham_total_emails_train/len(X_train))
spam_log_prior = np.log(spam_total_emails_train/len(X_train))
# ====================================================

Обчисліть правдоподібності (likelihood) для кожного слова. Також, застосуйте згладжування Лапласа, щоб уникнути ділення на нуль.

Створимо порожні вектори $\log{\phi_{word \, | \, ham}}$ та $\log{\phi_{word \, | \, spam}}$ і заповнимо їх для кожного слова зі словника.

In [38]:
ham_log_phi = np.zeros(len(vocab), dtype="float64")
spam_log_phi = np.zeros(len(vocab), dtype="float64")

In [39]:
# ham_word_counts = np.zeros(len(vocab))
# spam_word_counts = np.zeros(len(vocab))

In [40]:
# =============== TODO: Your code here ===============
# Compute log phi(word | class) for each word in the vocabulary.
# Fill out the `ham_log_phi` and `spam_log_phi` arrays below.
ham_log_phi.fill(0)
spam_log_phi.fill(0)

for i in range(len(X_train)):
    for word in X_train[i]:
        if (y_train[i] == 1):
            spam_log_phi[int(word)] += 1 
        else:
            ham_log_phi[int(word)] += 1 
    
spam_log_phi=np.log((spam_log_phi+1)/(spam_total_words_train+len(vocab)))
ham_log_phi=np.log((ham_log_phi+1)/(ham_total_words_train+len(vocab)))

# ====================================================

## Передбачення

Реалізуйте функцію передбачення. Пригадайте, що знаменник $P(words)$ — один і той самий для обох класів, тому для передбачення його можна проігнорувати.

In [41]:
def predict(X):
    # =============== TODO: Your code here ===============
    # Implement the prediction of target classes, given
    # a feature dataset X. You should return a response
    # vector containing n {0, 1} values, where n is the
    # number of examples in X.
    
    y_hat = np.zeros(len(X))
    
    for i in range(len(X)):
        spam_log_prob = spam_log_prior
        ham_log_prob = ham_log_prior
        for word in X[i]:
            spam_log_prob += (spam_log_phi[int(word)])
            ham_log_prob += (ham_log_phi[int(word)])

        if (spam_log_prob >= ham_log_prob):
            y_hat[i] = 1
        else:
            y_hat[i] = 0            
    return y_hat
# ====================================================

## Оцінка точності передбачення

In [42]:
pred_train = predict(X_train)
pred_test = predict(X_test)

In [43]:
accuracy_train = 1 - np.sum(pred_train != y_train) / len(y_train)
accuracy_test = 1 - np.sum(pred_test != y_test) / len(y_test)

In [44]:
print("Training accuracy:   {0:.3f}%".format(accuracy_train * 100))
print("Test accuracy:       {0:.3f}%".format(accuracy_test * 100))

Training accuracy:   97.654%
Test accuracy:       97.477%
